In [1]:
# doodleverse_utils Written by Dr Daniel Buscombe, Marda Science LLC
from doodleverse_utils.prediction_imports import *
from doodleverse_utils.imports import *
#=======================================================
# This script was written by Sharon Fitzpatrick
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from keras.metrics import BinaryAccuracy, FalseNegatives, FalsePositives, TrueNegatives,TruePositives, Precision, Recall, AUC

USE_GPU = False

if USE_GPU == True:
   ##use the first available GPU
   os.environ['CUDA_VISIBLE_DEVICES'] = '0' #'1'
else:
   ## to use the CPU (not recommended):
   os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

#suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

weights='C:/1_USGS/CoastSeg/repos/6_sniffer-classifer/Sniffer-Classifier/Dan_model/weights/resunet/ct_ortho_all_water_768.h5'


Version:  2.8.0
Eager mode:  True
Version:  2.8.0
Eager mode:  True
GPU name:  []
Num GPUs Available:  0


In [2]:
W=[]
W.append(weights)

M= []; C=[]; T = []
for counter,weights in enumerate(W):
    configfile = weights.replace('.h5','.json').replace('weights', 'config')
    with open(configfile) as f:
        config = json.load(f)
    for k in config.keys():
        expression=k+'=config["'+k+'"]'
        exec(k+'=config["'+k+'"]')
        # This exec dynamically creates all variables out of all the keys in config
    print('.....................................')
    print('Creating and compiling model {}...'.format(counter))
    if MODEL =='resunet':
        model =  custom_resunet((TARGET_SIZE[0], TARGET_SIZE[1], N_DATA_BANDS),
                        FILTERS,
                        nclasses=[NCLASSES+1 if NCLASSES==1 else NCLASSES][0],
                        kernel_size=(KERNEL,KERNEL),
                        strides=STRIDE,
                        dropout=DROPOUT,#0.1,
                        dropout_change_per_layer=DROPOUT_CHANGE_PER_LAYER,#0.0,
                        dropout_type=DROPOUT_TYPE,#"standard",
                        use_dropout_on_upsampling=USE_DROPOUT_ON_UPSAMPLING,#False,
                        )
    try:
        model = tf.keras.models.load_model(weights)
    except:
        # model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = [mean_iou, dice_coef])
        model.compile(optimizer = 'adam', loss = dice_coef_loss, metrics = [mean_iou, dice_coef])
        model.load_weights(weights)

        M.append(model)
        C.append(configfile)
        T.append(MODEL)

metadatadict = {}
metadatadict['model_weights'] = W
metadatadict['config_files'] = C
metadatadict['model_types'] = T

with open(configfile) as f:
    config = json.load(f)
config

.....................................
Creating and compiling model 0...


{'TARGET_SIZE': [768, 768],
 'MODEL': 'resunet',
 'NCLASSES': 1,
 'KERNEL': 9,
 'STRIDE': 2,
 'BATCH_SIZE': 8,
 'FILTERS': 6,
 'N_DATA_BANDS': 3,
 'DROPOUT': 0.1,
 'DROPOUT_CHANGE_PER_LAYER': 0.0,
 'DROPOUT_TYPE': 'standard',
 'USE_DROPOUT_ON_UPSAMPLING': False,
 'DO_TRAIN': True,
 'LOSS': 'cat',
 'PATIENCE': 10,
 'MAX_EPOCHS': 100,
 'VALIDATION_SPLIT': 0.7,
 'RAMPUP_EPOCHS': 20,
 'SUSTAIN_EPOCHS': 0.0,
 'EXP_DECAY': 0.9,
 'START_LR': 1e-07,
 'MIN_LR': 1e-07,
 'MAX_LR': 0.0001,
 'FILTER_VALUE': 0,
 'DOPLOT': True,
 'ROOT_STRING': 'ct_ortho_all_water_768',
 'USEMASK': False,
 'AUG_ROT': 5,
 'AUG_ZOOM': 0.05,
 'AUG_WIDTHSHIFT': 0.05,
 'AUG_HEIGHTSHIFT': 0.05,
 'AUG_HFLIP': True,
 'AUG_VFLIP': False,
 'AUG_LOOPS': 10,
 'AUG_COPIES': 5,
 'REMAP_CLASSES': {'0': 0,
  '1': 0,
  '2': 1,
  '3': 1,
  '4': 1,
  '5': 1,
  '6': 1,
  '7': 1,
  '8': 1,
  '9': 1,
  '10': 1,
  '11': 1,
  '12': 1}}

In [3]:
def getLayerIndexByName(model, layername):
    for idx, layer in enumerate(model.layers):
        if layer.name == layername:
            return idx

In [4]:
# When the model is initially loaded all of the layers are trainable
model.summary(show_trainable=True)

Model: "model"
_____________________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     Trainable  
 input_1 (InputLayer)           [(None, 768, 768, 3  0           []                               Y          
                                )]                                                                           
                                                                                                             
 conv2d (Conv2D)                (None, 768, 768, 6)  78          ['input_1[0][0]']                Y          
                                                                                                             
 batch_normalization (BatchNorm  (None, 768, 768, 6)  24         ['conv2d[0][0]']                 Y          
 alization)                                                                                              

In [5]:
# Get the last encoder layer in the base model
# Get the index of this layer so that we can examine the output
layer_name="conv2d_13"
idx=getLayerIndexByName(model,layer_name)
print(f"index of layer {layer_name}: {idx}")
# Examine the configuration of the add_4 layer
print(model.layers[idx].get_config(),"\n")
# Examine the output of the add_4 layer
print(model.layers[idx].output)
# Remove all the encoding layers after the add_4 layer
cut_base_model=Model(inputs=model.inputs,outputs=model.layers[idx].output)

index of layer conv2d_13: 55
{'name': 'conv2d_13', 'trainable': True, 'dtype': 'float32', 'filters': 96, 'kernel_size': (9, 9), 'strides': (1, 1), 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None} 

KerasTensor(type_spec=TensorSpec(shape=(None, 48, 48, 96), dtype=tf.float32, name=None), name='conv2d_13/BiasAdd:0', description="created by layer 'conv2d_13'")


In [6]:
# See all the layers in the cut base model to false, so that the weights will not be modified later
cut_base_model.summary(show_trainable=True)

Model: "model_1"
_____________________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     Trainable  
 input_1 (InputLayer)           [(None, 768, 768, 3  0           []                               Y          
                                )]                                                                           
                                                                                                             
 conv2d (Conv2D)                (None, 768, 768, 6)  78          ['input_1[0][0]']                Y          
                                                                                                             
 batch_normalization (BatchNorm  (None, 768, 768, 6)  24         ['conv2d[0][0]']                 Y          
 alization)                                                                                            

In [7]:
# Keep the base_model for transfer learning
cut_base_model.trainable=False
cut_base_model.compile(optimizer = 'adam', loss = keras.losses.BinaryCrossentropy())
# Validate that the trainable attribute of the batch_normalization layer is set to false
cut_base_model.get_layer("batch_normalization_12").trainable

False

In [8]:
# The base model requires that inputs have the shape (None,768,768,3)
img_shape = (768, 768)
inputs=keras.Input(img_shape+(3,))
inputs

<KerasTensor: shape=(None, 768, 768, 3) dtype=float32 (created by layer 'input_2')>

In [16]:
# Append the new classifying layers to the base model
x=cut_base_model.output
x= keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(64,activation='relu')(x)
x = keras.layers.Dense(1,activation="sigmoid")(x)

new_model = Model(inputs=cut_base_model.input,outputs=x)

In [17]:
new_model.summary(show_trainable=True)

Model: "model_3"
_____________________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     Trainable  
 input_1 (InputLayer)           [(None, 768, 768, 3  0           []                               N          
                                )]                                                                           
                                                                                                             
 conv2d (Conv2D)                (None, 768, 768, 6)  78          ['input_1[0][0]']                N          
                                                                                                             
 batch_normalization (BatchNorm  (None, 768, 768, 6)  24         ['conv2d[0][0]']                 N          
 alization)                                                                                            

In [18]:
metrics=[TruePositives(name='tp'),FalsePositives(name='fp'), TrueNegatives(name='tn'), FalseNegatives(name='fn'), BinaryAccuracy(name="BinaryAccuracy"), Precision(name="precision"), Recall(name="recall"),AUC(name="AUC")]
new_model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=metrics)


In [12]:
new_model.summary(show_trainable=True)

Model: "model_2"
_____________________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     Trainable  
 input_1 (InputLayer)           [(None, 768, 768, 3  0           []                               N          
                                )]                                                                           
                                                                                                             
 conv2d (Conv2D)                (None, 768, 768, 6)  78          ['input_1[0][0]']                N          
                                                                                                             
 batch_normalization (BatchNorm  (None, 768, 768, 6)  24         ['conv2d[0][0]']                 N          
 alization)                                                                                            

In [13]:
def save_model_data(model_name:str,model:'keras.engine.functional.Functional'):
    # Create a directory to store all the model data
    model_save_path=os.getcwd()+os.sep+"models"
    if not os.path.exists(model_save_path):
        os.mkdir(model_save_path)
    model_save_path=os.getcwd()+os.sep+"models"+os.sep+model_name
    if os.path.exists(model_save_path):
        raise FileExistsError(f"This folder '{model_name}' already exists: {model_save_path}.\n Please rename your model.")
    else:
        os.mkdir(model_save_path)
    # Save the actual model
    model.save(model_save_path+os.sep+"model")

In [19]:
# Enter the name of your model
model_name="transfer_model_v_2_1"
# This might take a few seconds
save_model_data(model_name,new_model)

INFO:tensorflow:Assets written to: c:\1_USGS\CoastSeg\repos\6_sniffer-classifer\Sniffer-Classifier\models\transfer_model_v_2_1\model\assets
